In [247]:
# Imported required libraries for the cleaning .
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [248]:
df = pd.read_csv('Messy_Employee_dataset.csv') # Imported the dataset.

In [249]:
df.shape # Quickly checked the dataset dimensions so we get an overall idea of the size of the data we are dealing with.

(1020, 12)

In [250]:
df # Skim the dataset.
'''
    Looking at the dataset I found out that most of the rows in Age column seem missing also 
the phone numbers seem to be having a - symbol misplaced at the start . 

    The dataset has many columns that can be used for prediction like : Salary,Performance_Score and Status.
'''


'\n    Looking at the dataset I found out that most of the rows in Age column seem missing also \nthe phone numbers seem to be having a - symbol misplaced at the start . \n\n    The dataset has many columns that can be used for prediction like : Salary,Performance_Score and Status.\n'

In [251]:
df.sample(5) # Sample returns a sample from the whole dataframe (n = no of samples to return).

,Employee_ID,First_Name,Last_Name,Age,Department_Region,Status,Join_Date,Salary,Email,Phone,Performance_Score,Remote_Work
49,EMP1049,Grace,Miller,NaN,DevOps-New York,Active,11/5/2024,61375.12,grace.miller@example.com,-1227770792,Excellent,True
753,EMP1753,Frank,Johnson,25.0,DevOps-Florida,Active,10/25/2022,109229.68,frank.johnson@example.com,-8335090253,Poor,True
848,EMP1848,Frank,Davis,30.0,HR-Illinois,Active,10/9/2023,50593.33,frank.davis@example.com,-1880819047,Excellent,True
46,EMP1046,Grace,Garcia,NaN,Cloud Tech-Texas,Active,12/13/2024,98371.37,grace.garcia@example.com,-7499564121,Excellent,False
2,EMP1002,Alice,Jones,NaN,Admin-Nevada,Pending,12/7/2023,88145.90,alice.jones@example.com,-5596363211,Good,True


In [252]:
df.info() 
# Here found out that there are many irregularities in the dataset , which were not visible at the surface.
# First of all mostly all the columns are objects which is a bad sign , cause ML algorithms are fed with numbers not objects,
# we have to thus device strategy to convert the appropriate features into numeric type accordingly.

# Also there were two columns with missing value : Age,Salary
# Salary had few missing values but Age had a lot of missing values 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Employee_ID        1020 non-null   object 
 1   First_Name         1020 non-null   object 
 2   Last_Name          1020 non-null   object 
 3   Age                809 non-null    float64
 4   Department_Region  1020 non-null   object 
 5   Status             1020 non-null   object 
 6   Join_Date          1020 non-null   object 
 7   Salary             996 non-null    float64
 8   Email              1020 non-null   object 
 9   Phone              1020 non-null   int64  
 10  Performance_Score  1020 non-null   object 
 11  Remote_Work        1020 non-null   bool   
dtypes: bool(1), float64(2), int64(1), object(8)
memory usage: 88.8+ KB


In [253]:
# After looking at the numbers I decided that:
#   First I will apply all the transformations and preprocessing steps to other columns.
#   Then we will use the new columns to predict the Age and Salary using KNN Imputer.
df.isna().sum()

Employee_ID            0
First_Name             0
Last_Name              0
Age                  211
Department_Region      0
Status                 0
Join_Date              0
Salary                24
Email                  0
Phone                  0
Performance_Score      0
Remote_Work            0
dtype: int64

In [254]:
df['Age'].unique()  
#    A quick check on the age column and this was basically the step I decided to go with KNN .

#   If u look properly the numbers here are divided into bins and not continous , 
# if we carelessly impute these with any simple imputing methods we could loss valuable Information.

#   So to deal with this I chose KNN as the best as it uses the other columns as the input to predict the values of Unknown values .

array([25., nan, 40., 30., 35.])

In [255]:
# Firstly I made a seperate column for Department and Region using string split method
# This will be further followed by using these information to encode into columns (OneHot) and we are gonna use it for calculating other values while doing KNN.

df['Department'] = df['Department_Region'].str.split('-').str[0]
df['Region'] = df['Department_Region'].str.split('-').str[1]

In [256]:
# The phone number was weirdly having a - symbol , so I decided to get rid of it to make it look clear.
# Note : I did not convert phone number to number dtype as this information is not necessary . 
# Even if we further plot graphs we dont need to use this many big values , as the information is redundant for us .

df['Phone'] = df['Phone'].astype(str).str.replace("-","",regex=False)

In [257]:
# Convert the date to type datetime , as this will enable many datetime functions for us.
df['Join_Date'] = pd.to_datetime(df['Join_Date'])

In [258]:
# The Remote work column is already binary in our case so we just convert them to binary digits.
df['Remote_Work'] =  df['Remote_Work'].astype(int)

In [259]:
df['Performance_Score'].unique() # Checking the performance score unique values 
# I decided to encode these values Ordinally using a map because that way we will have a heirarchy ,
#  so ML can clearly predict which score carrys higher weight respectively.

array(['Average', 'Excellent', 'Good', 'Poor'], dtype=object)

In [260]:
# Mapping using common sense
perfo_map = {
    'Average' : 2 ,
    'Excellent' : 4,
    'Good' : 3,
    'Poor' : 1 
}

In [261]:
df['Performance_Score'] = df['Performance_Score'].map(perfo_map) # Used the map function to map the values for the column.

In [262]:
# Now for the status we will apply OneHotEncoding as there is no order .
from sklearn.preprocessing import OneHotEncoder

In [263]:
# As we dont need sparce output I will make it false , also
# handle unknown is given the value ignore to handle values that are not listed in the column now.
# Basically it is a prevention step so that if in future any new unknown labels are added then it should be handled in the pipeline.
onc = OneHotEncoder(sparse_output=False,handle_unknown='ignore') 

In [264]:
# Fit transform the column accordingly.
# The datatype return is in float so I transformed it into int to reduce space
df['Status'] = onc.fit_transform(df[['Status']]).astype(int)

In [265]:
df # A quick check on the dataset
# Note : We must always keep a check of whats happening in the dataset and check as much time as we want.

,Employee_ID,First_Name,Last_Name,Age,Department_Region,Status,Join_Date,Salary,Email,Phone,Performance_Score,Remote_Work,Department,Region
0,EMP1000,Bob,Davis,25.0,DevOps-California,1,2021-04-02,59767.65,bob.davis@example.com,1651623197,2,1,DevOps,California
1,EMP1001,Bob,Brown,NaN,Finance-Texas,1,2020-07-10,65304.66,bob.brown@example.com,1898471390,4,1,Finance,Texas
2,EMP1002,Alice,Jones,NaN,Admin-Nevada,0,2023-12-07,88145.90,alice.jones@example.com,5596363211,3,1,Admin,Nevada
3,EMP1003,Eva,Davis,25.0,Admin-Nevada,0,2021-11-27,69450.99,eva.davis@example.com,3476490784,3,1,Admin,Nevada
4,EMP1004,Frank,Williams,25.0,Cloud Tech-Florida,1,2022-01-05,109324.61,frank.williams@example.com,1586734256,1,0,Cloud Tech,Florida
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,EMP2015,David,Miller,30.0,HR-California,1,2023-08-19,NaN,david.miller@example.com,3546212759,3,1,HR,California
1016,EMP2016,David,Johnson,30.0,Cloud Tech-Texas,0,2021-11-07,100215.06,david.johnson@example.com,2508261122,3,1,Cloud Tech,Texas
1017,EMP2017,Charlie,Williams,40.0,Finance-New York,1,2023-10-04,114587.11,charlie.williams@example.com,1261632487,2,0,Finance,New York
1018,EMP2018,Alice,Garcia,30.0,HR-Florida,0,2024-12-16,71318.79,alice.garcia@example.com,8995729892,3,1,HR,Florida


In [266]:
df.info() # A quick look at the data.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Employee_ID        1020 non-null   object        
 1   First_Name         1020 non-null   object        
 2   Last_Name          1020 non-null   object        
 3   Age                809 non-null    float64       
 4   Department_Region  1020 non-null   object        
 5   Status             1020 non-null   int64         
 6   Join_Date          1020 non-null   datetime64[ns]
 7   Salary             996 non-null    float64       
 8   Email              1020 non-null   object        
 9   Phone              1020 non-null   object        
 10  Performance_Score  1020 non-null   int64         
 11  Remote_Work        1020 non-null   int64         
 12  Department         1020 non-null   object        
 13  Region             1020 non-null   object        
dtypes: datet

In [267]:
df.columns # At this stage I have not removed any unnecessary columns , I have decided to remove it later .

Index(['Employee_ID', 'First_Name', 'Last_Name', 'Age', 'Department_Region',
       'Status', 'Join_Date', 'Salary', 'Email', 'Phone', 'Performance_Score',
       'Remote_Work', 'Department', 'Region'],
      dtype='object')

In [268]:
# Now according to some analysis I found out that Date year is one of the crucial pieces of information and we can use it in our KNN imputing.
df['Join_Year'] = df['Join_Date'].dt.year

In [269]:
# To be safe , I have justed copied the whole dataframe into a backup one.
df1 = df[:]
# As we have seperate columns for the Department and Region we no longer need this column.
df = df.drop(columns=['Department_Region']) 

In [270]:
# Now this is a subset dataset where we will be performing all the processing steps further for KNN and other remaining,
# steps seperately.

# This is to make a dataframe which only has numeric type data only.
X = df.drop(columns=['Employee_ID', 'First_Name', 'Last_Name', 'Email', 'Phone', 'Join_Date'])

In [271]:
X.info() # Verified the columns present.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                809 non-null    float64
 1   Status             1020 non-null   int64  
 2   Salary             996 non-null    float64
 3   Performance_Score  1020 non-null   int64  
 4   Remote_Work        1020 non-null   int64  
 5   Department         1020 non-null   object 
 6   Region             1020 non-null   object 
 7   Join_Year          1020 non-null   int32  
dtypes: float64(2), int32(1), int64(3), object(2)
memory usage: 59.9+ KB


In [272]:
# Here we have made anothe instantiation of the OneHotEncoder .
# I decided that we would use the department and region columns ane onehotencode them.
# This grealty improves the models accuracy for prediction while applying KNN

onc = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

In [273]:
# Selected the columns and fit transformed them.
# Created a temp variable for storing these transformations into
X_dept_encoded = onc.fit_transform(df[['Department','Region']])

In [274]:
# Storing the column names for this values in another temp
X_dept_encoded_cols = onc.get_feature_names_out(['Department','Region'])

In [275]:
# Made a dataframe from the temp variables so we can concatinate with X.
X_dept = pd.DataFrame(X_dept_encoded,columns=X_dept_encoded_cols)

In [276]:
X_dept # Quick check 

,Department_Admin,Department_Cloud Tech,Department_DevOps,Department_Finance,Department_HR,Department_Sales,Region_California,Region_Florida,Region_Illinois,Region_Nevada,Region_New York,Region_Texas
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1015,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1016,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1017,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1018,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [277]:
# As the Department and Region column is not needed anymore we wil drop it (another reason is we can not have any objects in X).
X = X.drop(columns=['Department','Region'])

In [278]:
X = pd.concat([X,X_dept],axis=1) # Concatinating both

In [279]:
X # Quick skim

,Age,Status,Salary,Performance_Score,Remote_Work,Join_Year,Department_Admin,Department_Cloud Tech,Department_DevOps,Department_Finance,Department_HR,Department_Sales,Region_California,Region_Florida,Region_Illinois,Region_Nevada,Region_New York,Region_Texas
0,25.0,1,59767.65,2,1,2021,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,NaN,1,65304.66,4,1,2020,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,NaN,0,88145.90,3,1,2023,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,25.0,0,69450.99,3,1,2021,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,25.0,1,109324.61,1,0,2022,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,30.0,1,NaN,3,1,2023,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1016,30.0,0,100215.06,3,1,2021,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1017,40.0,1,114587.11,2,0,2023,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1018,30.0,0,71318.79,3,1,2024,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [280]:
X.info() # Checking for if there are any irrregualrites left other than age and salary.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Age                    809 non-null    float64
 1   Status                 1020 non-null   int64  
 2   Salary                 996 non-null    float64
 3   Performance_Score      1020 non-null   int64  
 4   Remote_Work            1020 non-null   int64  
 5   Join_Year              1020 non-null   int32  
 6   Department_Admin       1020 non-null   float64
 7   Department_Cloud Tech  1020 non-null   float64
 8   Department_DevOps      1020 non-null   float64
 9   Department_Finance     1020 non-null   float64
 10  Department_HR          1020 non-null   float64
 11  Department_Sales       1020 non-null   float64
 12  Region_California      1020 non-null   float64
 13  Region_Florida         1020 non-null   float64
 14  Region_Illinois        1020 non-null   float64
 15  Regi

In [281]:
# Now the plan is to first scale the values perform KNN and then inverse scale into original
from sklearn.preprocessing import StandardScaler

In [282]:
scaler = StandardScaler() # Instantiate the StandardScaler

In [283]:
X_scalled = scaler.fit_transform(X) # Fit transform X

In [284]:
from sklearn.impute import KNNImputer # Import the KNNImputer from sklearn.impute 

In [285]:
knn = KNNImputer(n_neighbors=5,weights='distance') 
# The neighbors are set to 5 (standard) and weights as distance (to calculate the values using distance from point to point (nearest k))

In [286]:
knn_arr = knn.fit_transform(X_scalled) # Fit transform it and store it in a temp variable.

In [287]:
knn_arr = scaler.inverse_transform(knn_arr) # Now reverse scale it so we get the original scale back.

In [288]:
imputed = pd.DataFrame(knn_arr,columns=X.columns) # Now making this into an dataframe 

In [289]:
imputed['Age'] = (imputed['Age']/5).round() * 5 
# This is one of the most crucial step which i missed earlier , 
# as there are already given intervals we must stick with them and need to create new ones 
# so as there is a difference of 5 in each interval we will stick with rounding to nearest 5.

# The age here is in decimals so we round it accordingly so that it is rounded to nearest 5 .

In [290]:
imputed # Quick look on the imputed value

,Age,Status,Salary,Performance_Score,Remote_Work,Join_Year,Department_Admin,Department_Cloud Tech,Department_DevOps,Department_Finance,Department_HR,Department_Sales,Region_California,Region_Florida,Region_Illinois,Region_Nevada,Region_New York,Region_Texas
0,25.0,1.0,59767.650000,2.0,1.0,2021.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,30.0,1.0,65304.660000,4.0,1.0,2020.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,35.0,0.0,88145.900000,3.0,1.0,2023.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,25.0,0.0,69450.990000,3.0,1.0,2021.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,25.0,1.0,109324.610000,1.0,0.0,2022.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,30.0,1.0,85628.995756,3.0,1.0,2023.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1016,30.0,0.0,100215.060000,3.0,1.0,2021.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1017,40.0,1.0,114587.110000,2.0,0.0,2023.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1018,30.0,0.0,71318.790000,3.0,1.0,2024.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [291]:
df 

,Employee_ID,First_Name,Last_Name,Age,Status,Join_Date,Salary,Email,Phone,Performance_Score,Remote_Work,Department,Region,Join_Year
0,EMP1000,Bob,Davis,25.0,1,2021-04-02,59767.65,bob.davis@example.com,1651623197,2,1,DevOps,California,2021
1,EMP1001,Bob,Brown,NaN,1,2020-07-10,65304.66,bob.brown@example.com,1898471390,4,1,Finance,Texas,2020
2,EMP1002,Alice,Jones,NaN,0,2023-12-07,88145.90,alice.jones@example.com,5596363211,3,1,Admin,Nevada,2023
3,EMP1003,Eva,Davis,25.0,0,2021-11-27,69450.99,eva.davis@example.com,3476490784,3,1,Admin,Nevada,2021
4,EMP1004,Frank,Williams,25.0,1,2022-01-05,109324.61,frank.williams@example.com,1586734256,1,0,Cloud Tech,Florida,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,EMP2015,David,Miller,30.0,1,2023-08-19,NaN,david.miller@example.com,3546212759,3,1,HR,California,2023
1016,EMP2016,David,Johnson,30.0,0,2021-11-07,100215.06,david.johnson@example.com,2508261122,3,1,Cloud Tech,Texas,2021
1017,EMP2017,Charlie,Williams,40.0,1,2023-10-04,114587.11,charlie.williams@example.com,1261632487,2,0,Finance,New York,2023
1018,EMP2018,Alice,Garcia,30.0,0,2024-12-16,71318.79,alice.garcia@example.com,8995729892,3,1,HR,Florida,2024


In [292]:
df.drop(columns=['Join_Year'],inplace=True) # Now as we dont need the Join_year anymore in original dataset we will remove it .

In [ ]:
df.info() # Now we will replace these values with the new values from imputed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Employee_ID        1020 non-null   object        
 1   First_Name         1020 non-null   object        
 2   Last_Name          1020 non-null   object        
 3   Age                809 non-null    float64       
 4   Status             1020 non-null   int64         
 5   Join_Date          1020 non-null   datetime64[ns]
 6   Salary             996 non-null    float64       
 7   Email              1020 non-null   object        
 8   Phone              1020 non-null   object        
 9   Performance_Score  1020 non-null   int64         
 10  Remote_Work        1020 non-null   int64         
 11  Department         1020 non-null   object        
 12  Region             1020 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(7)
memory us

In [ ]:
# Replacing the values from both dataframe.
df['Age'] = imputed['Age']
df['Salary'] = imputed['Salary']

In [ ]:
df.isnull().sum() # A quick recheck.

Employee_ID          0
First_Name           0
Last_Name            0
Age                  0
Status               0
Join_Date            0
Salary               0
Email                0
Phone                0
Performance_Score    0
Remote_Work          0
Department           0
Region               0
dtype: int64

In [ ]:
df.columns 
# The below step is done just done to improve reader interpretibility by rearranging the columns.

Index(['Employee_ID', 'First_Name', 'Last_Name', 'Age', 'Status', 'Join_Date',
       'Salary', 'Email', 'Phone', 'Performance_Score', 'Remote_Work',
       'Department', 'Region'],
      dtype='object')

In [297]:
new_order = ['Employee_ID', 'First_Name', 'Last_Name',
       'Department', 'Region' ,'Age', 'Status', 'Join_Date',
       'Salary', 'Email', 'Phone', 'Performance_Score', 'Remote_Work']

In [298]:
df = df[new_order]

In [299]:
df

,Employee_ID,First_Name,Last_Name,Department,Region,Age,Status,Join_Date,Salary,Email,Phone,Performance_Score,Remote_Work
0,EMP1000,Bob,Davis,DevOps,California,25.0,1,2021-04-02,59767.650000,bob.davis@example.com,1651623197,2,1
1,EMP1001,Bob,Brown,Finance,Texas,30.0,1,2020-07-10,65304.660000,bob.brown@example.com,1898471390,4,1
2,EMP1002,Alice,Jones,Admin,Nevada,35.0,0,2023-12-07,88145.900000,alice.jones@example.com,5596363211,3,1
3,EMP1003,Eva,Davis,Admin,Nevada,25.0,0,2021-11-27,69450.990000,eva.davis@example.com,3476490784,3,1
4,EMP1004,Frank,Williams,Cloud Tech,Florida,25.0,1,2022-01-05,109324.610000,frank.williams@example.com,1586734256,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,EMP2015,David,Miller,HR,California,30.0,1,2023-08-19,85628.995756,david.miller@example.com,3546212759,3,1
1016,EMP2016,David,Johnson,Cloud Tech,Texas,30.0,0,2021-11-07,100215.060000,david.johnson@example.com,2508261122,3,1
1017,EMP2017,Charlie,Williams,Finance,New York,40.0,1,2023-10-04,114587.110000,charlie.williams@example.com,1261632487,2,0
1018,EMP2018,Alice,Garcia,HR,Florida,30.0,0,2024-12-16,71318.790000,alice.garcia@example.com,8995729892,3,1


In [ ]:
# A quick verification to ensure everything was done right.
for i in df.columns:
    print(pd.DataFrame(df[i].unique()))

            0
0     EMP1000
1     EMP1001
2     EMP1002
3     EMP1003
4     EMP1004
...       ...
1015  EMP2015
1016  EMP2016
1017  EMP2017
1018  EMP2018
1019  EMP2019

[1020 rows x 1 columns]
         0
0      Bob
1    Alice
2      Eva
3    Frank
4  Charlie
5    David
6    Heidi
7    Grace
          0
0     Davis
1     Brown
2     Jones
3  Williams
4    Garcia
5   Johnson
6    Miller
7     Smith
            0
0      DevOps
1     Finance
2       Admin
3  Cloud Tech
4       Sales
5          HR
            0
0  California
1       Texas
2      Nevada
3     Florida
4    New York
5    Illinois
      0
0  25.0
1  30.0
2  35.0
3  40.0
   0
0  1
1  0
             0
0   2021-04-02
1   2020-07-10
2   2023-12-07
3   2021-11-27
4   2022-01-05
..         ...
755 2021-07-02
756 2022-12-05
757 2024-09-24
758 2023-08-16
759 2021-01-28

[760 rows x 1 columns]
                  0
0      59767.650000
1      65304.660000
2      88145.900000
3      69450.990000
4     109324.610000
...             ...
996  

In [ ]:
df.to_csv('cleaned_Messy_Employee_dataset.csv') # Finally export the dataset as suitable format 